# Forward/Inverse Kinematics Examples

Kevin Walchko
Created: 10 July 2017

---

# Denavit-Hartenberg (DH)

Development of the DH forward kinematics for a robot. We will use the DH method to develop the symbolic equations and use python to simplify them.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d8/DHParameter.png/609px-DHParameter.png)



In [1]:
%matplotlib inline

In [44]:
from __future__ import print_function
from __future__ import division
import numpy as np
from sympy import symbols, sin, cos, pi, simplify, trigsimp
from math import radians as d2r
from math import degrees as r2d
from math import atan2, sqrt, acos, fabs
from pprint import pprint

# Kinematics

Robot arms and legs are hard to control (lots of math), but required for most robotic applications.

- [darpa robot challenge](https://www.youtube.com/watch?v=diaZFIUBMBQ)
- [darpa robot fails](https://youtu.be/wX0KagJ1du8)

## Coordinate Frames

To help, we are going to follow a methodical way to assign coordinate frames to a robot. An example (sort of) is shown below:

![](dh_pics/kuka-kr270.png)

The KR270 has 6 joints and is a standard industrial type robot for building cars or what not.

## Numerical Forward Kinematics

---

The following class follows the modified [DH convention](https://en.wikipedia.org/wiki/Denavit%E2%80%93Hartenberg_parameters). Where:

| Parameter | Definition|
|-----------|-----------|
| $\alpha_i$   | angle about common normal, from old z axis to new z axis (twist)|
| $d_i$        | offset along previous z to the common normal |
| $\theta_i$   | angle about previous z, from old x to new x |
| $a_i$        | length of the common normal. Assuming a revolute joint, this is the radius about previous z. |

Now the transformation matrix ($T$), using modified DH parameters, is given by:

$$ T = R_x(\alpha_{n-1}) D_x(a_{n-1}) R_z(\theta_n) D_z(d_n) $$

We can then turn this into a python class as follows:

In [102]:
class mDH(object):
    """
    This uses the modified DH parameters
    see Craig, eqn (3.6)
    """
    def __init__(self):
        pass

    def fk(self, params):
        t = np.eye(4)
        for p in params:
            t = t.dot(self.makeT(*p))
        return t

    def makeT(self, a, alpha, d, theta):
#         alpha = d2r(alpha)
#         theta = d2r(theta)
        return np.array([  # classic DH
            [           cos(theta),           -sin(theta),           0,             a],
            [sin(theta)*cos(alpha), cos(theta)*cos(alpha), -sin(alpha), -d*sin(alpha)],
            [sin(theta)*sin(alpha), cos(theta)*sin(alpha),  cos(alpha),  d*cos(alpha)],
            [                    0,                     0,           0,             1]
        ])


def printT(tt):
    R = tt[0:3,0:3]
    D = tt[0:3, 3]
    print('-'*30)
    print('Position:')
    print('  x:', D[0])
    print('  y:', D[1])
    print('  z:', D[2])
    # R(n, o, a)
    print('-'*30)
    print('Orientation')
    print('  nx:', R[0,0])
    print('  ny:', R[0,1])
    print('  nz:', R[0,2])
    print('')
    print('  ox:', R[1,0])
    print('  oy:', R[1,1])
    print('  oz:', R[1,2])
    print('')
    print('  ax:', R[2,0])
    print('  ay:', R[2,1])
    print('  az:', R[2,2])

    
def simplifyT(tt):
    
    for i, row in enumerate(tt):
        for j, col in enumerate(row):
            tt[i,j] = simplify(col)
    return tt

## Symbolically Solving Forward Kinematics

In [31]:
t1, t2, t3 = symbols('t1 t2 t3')
L1, L2, L3 = symbols('L1 L2 L3')

In [4]:
def eval(f, inputs):
    """
    This allows you to simplify the trigonomic mess that kinematics can
    create and also substitute in some inputs in the process
    """
    c = []
    for i in f:
        r = []
        for j in i:
            tmp = j
#             tmp = j.subs(inputs)
            # substitute in any in puts if entered
#             tmp = tmp.evalf()
            # use python symbolic toolbox to simplify the trig mess above 
            tmp = simplify(tmp)
            r.append(tmp)
        c.append(r)
    return np.array(c)

The parameters are:

| i |$a_i$        | $\alpha_i$   | $d_i$   | $theta_i$  |
|---|-------------|--------------|---------|------------|
| 1 | 0           | 0            | 0       | $\theta_1$ |
| 2 | $L_1$       | 0            | 0       | $\theta_2$ |
| 3 | $L_2$       | 0            | 0       | $\theta_3$ |


In [55]:
t1, t2, t3 = symbols('t1 t2 t3')
L1, L2, L3 = symbols('L1 L2 L3')
# a, alpha, d, theta
params = [
    [ 0, 0, 0, t1],  # DH parameters for the first link (i.e., the first row in the table above)
    [L1, 0, 0, t2],  # 2nd link
    [L2, 0, 0, t3]   # 3rd link
]
dh = mDH()
t = dh.fk(params)
t = eval(t,[])

In [56]:
R = t[0:3,0:3]
print('Rotation Matrix:')
pprint(R)
D = t[0:3, 3]
print('Position:')
print('  x:', D[0])
print('  y:', D[1])
print('  z:', D[2])

Rotation Matrix:
array([[1.0*cos(t1 + t2 + t3), -1.0*sin(t1 + t2 + t3), 0],
       [1.0*sin(t1 + t2 + t3), 1.0*cos(t1 + t2 + t3), 0],
       [0, 0, 1.00000000000000]], dtype=object)
Position:
  x: 1.0*L1*cos(t1) + 1.0*L2*cos(t1 + t2)
  y: 1.0*L1*sin(t1) + 1.0*L2*sin(t1 + t2)
  z: 0


In [25]:
# now, if we want better performance, you can just use the above solution
# for our forward kinematics instead of building it each time we need it
# using the mDH class (which is a lot more computational)
def fk(t1, t2, d, L1, L2, L3):
    t1 = d2r(t1)  # python uses radians, and typicall we use degrees, so fix it
    t2 = d2r(t2)
    return np.array([
        1.0*L1*cos(t1) + 1.0*L2*cos(t1 + t2) + 1.0*L3*cos(t1 + t2),
        1.0*L1*sin(t1) + 1.0*L2*sin(t1 + t2) + 1.0*L3*sin(t1 + t2),
        0
    ])

# Inverse Kinematics


---


Assume the solution is this:

$$
\begin{bmatrix}
    c_{\gamma} & -s_{\gamma} & 0 & x \\
    s_{\gamma} & c_{\gamma} & 0 & y \\
    0 & 0 & 1 & 0 \\
    0 & 0 & 0 & 1 
\end{bmatrix}
$$

Note, that there is simplifications here to make reading/writing these equations easier, such as:

$$
\cos(\theta_1) = c_1 \\
\sin(\theta_1 + \theta_1) = s_{12}
$$

Now set the above matrix equal to it, (and following along in Craig, p 110) we get:

$$
c_{\gamma} = c_{123} \\
s_{\gamma} = s_{123} \\
x = L1*c_1+L2*c_{12} \\
y = L1*s_1+L1*s_{12}
$$

In [110]:
def trans(tt):
    # calculates the inverse of a tranformation matrix
    r = tt[0:3,0:3]
    d = tt[0:3, 3]
#     print('r', r.transpose())
    ret = np.eye(4, dtype=object)
    ret[0:3,0:3] = r.transpose()
#     rt = r.transpose()
#     for i in range(3):
#         for j in range(3):
#             ret[i,j] = rt[i,j]
    ret[0:3,3] = -r.transpose().dot(d)
    return ret

In [88]:
print(t)
print('-'*20)
print(trans(t))

[[1.0*cos(t1 + t2 + t3) -1.0*sin(t1 + t2 + t3) 0
  1.0*L1*cos(t1) + 1.0*L2*cos(t1 + t2)]
 [1.0*sin(t1 + t2 + t3) 1.0*cos(t1 + t2 + t3) 0
  1.0*L1*sin(t1) + 1.0*L2*sin(t1 + t2)]
 [0 0 1.00000000000000 0]
 [0 0 0 1.00000000000000]]
--------------------
[[1.0*cos(t1 + t2 + t3) 1.0*sin(t1 + t2 + t3) 0
  -1.0*(1.0*L1*sin(t1) + 1.0*L2*sin(t1 + t2))*sin(t1 + t2 + t3) - 1.0*(1.0*L1*cos(t1) + 1.0*L2*cos(t1 + t2))*cos(t1 + t2 + t3)]
 [-1.0*sin(t1 + t2 + t3) 1.0*cos(t1 + t2 + t3) 0
  -1.0*(1.0*L1*sin(t1) + 1.0*L2*sin(t1 + t2))*cos(t1 + t2 + t3) + 1.0*(1.0*L1*cos(t1) + 1.0*L2*cos(t1 + t2))*sin(t1 + t2 + t3)]
 [0 0 1.00000000000000 0]
 [0 0 0 1]]


# Test

In [48]:
L, d = symbols('L d')
# a, alpha, d, theta
params = [
    [ 0, 0, 0, 0]
]
dh = mDH()
t = dh.fk(params)
pprint(t)

array([[1.00000000000000, 0, 0, 0],
       [0, 1.00000000000000, 0, 0],
       [0, 0, 1.00000000000000, 0],
       [0, 0, 0, 1.00000000000000]], dtype=object)


In [72]:
m = np.array([[1,2],[3,4]])
print(m)
print(m.transpose())
print(m)

[[1 2]
 [3 4]]
[[1 3]
 [2 4]]
[[1 2]
 [3 4]]


In [75]:
m[0, 0:2] = np.array([5,5])
print(m)

[[5 5]
 [3 4]]


In [82]:
i = np.eye(4, dtype=object)
pprint(i)

array([[1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]], dtype=object)


## Lynx Motion AL5D

![](dh_pics/linx_motion_al5d.jpg)

In [112]:
t1, t2, t3, t4 = symbols('t1 t2 t3 t4')
a2, a3, a4, a5 = symbols('a2 a3 a4 a5')
ca, sa, px, py, pz = symbols('ca sa px py pz')

desired = np.array([[ca, 0, sa, px], [-sa, 0, ca, py], [0, -1, 0, pz], [0, 0, 0, 1]])

# a2 = 2.75   # base to shoulder
# a3 = 5.75   # shoulder to elbow
# a4 = 7.375  # elbow to wrist
# a5 = 3.375  # wrist to end effector

# put all angles in rads
# sybolic can't handle converting a symbol using d2r()
# a, alpha, d, theta
params = [
    [    0,    0,    a2, t1],
    [    0, pi/2,   0, t2],
    [   a3,    0,    0, t3],
    [   a4,    0,    0, t4],
    [   a5,    0,    0,  0]
]
dh = mDH()
al5d = dh.fk(params)
al5d = eval(al5d,[])
printT(al5d)

------------------------------
Position:
  x: 1.0*(a3*cos(t2) + a4*cos(t2 + t3) + a5*cos(t2 + t3 + t4))*cos(t1)
  y: 1.0*(a3*cos(t2) + a4*cos(t2 + t3) + a5*cos(t2 + t3 + t4))*sin(t1)
  z: 1.0*a2 + 1.0*a3*sin(t2) + 1.0*a4*sin(t2 + t3) + 1.0*a5*sin(t2 + t3 + t4)
------------------------------
Orientation
  nx: 1.0*cos(t1)*cos(t2 + t3 + t4)
  ny: -1.0*sin(t2 + t3 + t4)*cos(t1)
  nz: 1.0*sin(t1)

  ox: 1.0*sin(t1)*cos(t2 + t3 + t4)
  oy: -1.0*sin(t1)*sin(t2 + t3 + t4)
  oz: -1.0*cos(t1)

  ax: 1.0*sin(t2 + t3 + t4)
  ay: 1.0*cos(t2 + t3 + t4)
  az: 0


In [118]:
# a, alpha, d, theta
params = [
    [    0,    0,    a2, t1],
    [    0, pi/2,   0, t2]
]
dh = mDH()
t_12 = dh.fk(params)
t_12 = eval(t_12,[])
print(t_12)
tt_12 = trans(t_12)
a_35 = tt_12.dot(al5d)
a_35 = simplifyT(a_35)
printT(a_35)

d_35 = tt_12.dot(desired)
printT(d_35)

[[1.0*cos(t1)*cos(t2) -1.0*sin(t2)*cos(t1) 1.0*sin(t1) 0]
 [1.0*sin(t1)*cos(t2) -1.0*sin(t1)*sin(t2) -1.0*cos(t1) 0]
 [1.0*sin(t2) 1.0*cos(t2) 0 1.0*a2]
 [0 0 0 1.00000000000000]]
------------------------------
Position:
  x: 1.0*a3 + 1.0*a4*cos(t3) + 1.0*a5*cos(t3 + t4)
  y: 1.0*a4*sin(t3) + 1.0*a5*sin(t3 + t4)
  z: 0
------------------------------
Orientation
  nx: 1.0*cos(t3 + t4)
  ny: -1.0*sin(t3 + t4)
  nz: 0

  ox: 1.0*sin(t3 + t4)
  oy: 1.0*cos(t3 + t4)
  oz: 0

  ax: 0
  ay: 0
  az: 1.00000000000000
------------------------------
Position:
  x: -1.0*a2*sin(t2) + 1.0*px*cos(t1)*cos(t2) + 1.0*py*sin(t1)*cos(t2) + 1.0*pz*sin(t2)
  y: -1.0*a2*cos(t2) - 1.0*px*sin(t2)*cos(t1) - 1.0*py*sin(t1)*sin(t2) + 1.0*pz*cos(t2)
  z: 1.0*px*sin(t1) - 1.0*py*cos(t1)
------------------------------
Orientation
  nx: 1.0*ca*cos(t1)*cos(t2) - 1.0*sa*sin(t1)*cos(t2)
  ny: -1.0*sin(t2)
  nz: 1.0*ca*sin(t1)*cos(t2) + 1.0*sa*cos(t1)*cos(t2)

  ox: -1.0*ca*sin(t2)*cos(t1) + 1.0*sa*sin(t1)*sin(t2)
  oy: 

In [117]:
# a, alpha, d, theta
params = [
    [    0,    0,    a2, t1]
]
dh = mDH()
t_1 = dh.fk(params)
t_1 = eval(t_1,[])
print(t_1)
tt_1 = trans(t_1)
a_25 = tt_1.dot(al5d)
a_25 = simplifyT(a_25)
printT(a_25)

d_25 = tt_1.dot(desired)
printT(d_25)

[[1.0*cos(t1) -1.0*sin(t1) 0 0]
 [1.0*sin(t1) 1.0*cos(t1) 0 0]
 [0 0 1.00000000000000 1.0*a2]
 [0 0 0 1.00000000000000]]
------------------------------
Position:
  x: 1.0*a3*cos(t2) + 1.0*a4*cos(t2 + t3) + 1.0*a5*cos(t2 + t3 + t4)
  y: 0
  z: 1.0*a3*sin(t2) + 1.0*a4*sin(t2 + t3) + 1.0*a5*sin(t2 + t3 + t4)
------------------------------
Orientation
  nx: 1.0*cos(t2 + t3 + t4)
  ny: -1.0*sin(t2 + t3 + t4)
  nz: 0

  ox: 0
  oy: 0
  oz: -1.00000000000000

  ax: 1.0*sin(t2 + t3 + t4)
  ay: 1.0*cos(t2 + t3 + t4)
  az: 0
------------------------------
Position:
  x: 1.0*px*cos(t1) + 1.0*py*sin(t1)
  y: -1.0*px*sin(t1) + 1.0*py*cos(t1)
  z: -1.0*a2 + 1.0*pz
------------------------------
Orientation
  nx: 1.0*ca*cos(t1) - 1.0*sa*sin(t1)
  ny: 0
  nz: 1.0*ca*sin(t1) + 1.0*sa*cos(t1)

  ox: -1.0*ca*sin(t1) - 1.0*sa*cos(t1)
  oy: 0
  oz: 1.0*ca*cos(t1) - 1.0*sa*sin(t1)

  ax: 0
  ay: -1.00000000000000
  az: 0


In [119]:
# a, alpha, d, theta
params = [
    [    0,    0,    a2, t1],
    [    0, pi/2,   0, t2],
    [   a3,    0,    0, t3]
]
dh = mDH()
t_13 = dh.fk(params)
t_13 = eval(t_13,[])
# print(t_13)
tt_13 = trans(t_13)
a_45 = tt_13.dot(al5d)
a_45 = simplifyT(a_45)
printT(a_45)

d_45 = tt_13.dot(desired)
printT(d_45)

------------------------------
Position:
  x: 1.0*a4 + 1.0*a5*cos(t4)
  y: 1.0*a5*sin(t4)
  z: 0
------------------------------
Orientation
  nx: 1.0*cos(t4)
  ny: -1.0*sin(t4)
  nz: 0

  ox: 1.0*sin(t4)
  oy: 1.0*cos(t4)
  oz: 0

  ax: 0
  ay: 0
  az: 1.00000000000000
------------------------------
Position:
  x: -1.0*a3*sin(t1)**2*cos(t2)*cos(t2 + t3) - 1.0*a3*cos(t1)**2*cos(t2)*cos(t2 + t3) + 1.0*px*cos(t1)*cos(t2 + t3) + 1.0*py*sin(t1)*cos(t2 + t3) + 1.0*pz*sin(t2 + t3) - 1.0*(1.0*a2 + 1.0*a3*sin(t2))*sin(t2 + t3)
  y: 1.0*a3*sin(t1)**2*sin(t2 + t3)*cos(t2) + 1.0*a3*sin(t2 + t3)*cos(t1)**2*cos(t2) - 1.0*px*sin(t2 + t3)*cos(t1) - 1.0*py*sin(t1)*sin(t2 + t3) + 1.0*pz*cos(t2 + t3) - 1.0*(1.0*a2 + 1.0*a3*sin(t2))*cos(t2 + t3)
  z: 1.0*px*sin(t1) - 1.0*py*cos(t1)
------------------------------
Orientation
  nx: 1.0*ca*cos(t1)*cos(t2 + t3) - 1.0*sa*sin(t1)*cos(t2 + t3)
  ny: -1.0*sin(t2 + t3)
  nz: 1.0*ca*sin(t1)*cos(t2 + t3) + 1.0*sa*cos(t1)*cos(t2 + t3)

  ox: -1.0*ca*sin(t2 + t3)*cos(

# Puma

![](dh_pics/puma.png)

Craig, p 80

In [55]:
t1, t2, t3, t4, t5, t6 = symbols('t1 t2 t3 t4 t5 t6')
a2, a3, d3, d4 = symbols('a2 a3 d3 d4')
# a, alpha, d, theta
params = [
    [    0,     0,    0, t1],
    [    0, -pi/2,    0, t2],
    [   a2,     0,   d3, t3],
    [   a3, -pi/2,   d4, t4],
    [    0,    pi,    0, t5],
    [    0, -pi/2,    0, t6]
]
dh = mDH()
pumma = dh.fk(params)
pumma = eval(pumma,[])

In [56]:
R = pumma[0:3,0:3]
print('Rotation Matrix:')
pprint(R)
D = pumma[0:3, 3]
print('Position:')
print('  x:', D[0])
print('  y:', D[1])
print('  z:', D[2])

Rotation Matrix:
array([[ 1.0*(sin(t1)*sin(t4 - t5) + cos(t1)*cos(t2 + t3)*cos(t4 - t5))*cos(t6) - 1.0*sin(t6)*sin(t2 + t3)*cos(t1),
        -1.0*(sin(t1)*sin(t4 - t5) + cos(t1)*cos(t2 + t3)*cos(t4 - t5))*sin(t6) - 1.0*sin(t2 + t3)*cos(t1)*cos(t6),
        -1.0*sin(t1)*cos(t4 - t5) + 1.0*sin(t4 - t5)*cos(t1)*cos(t2 + t3)],
       [ 1.0*(sin(t1)*cos(t2 + t3)*cos(t4 - t5) - sin(t4 - t5)*cos(t1))*cos(t6) - 1.0*sin(t1)*sin(t6)*sin(t2 + t3),
        -1.0*(sin(t1)*cos(t2 + t3)*cos(t4 - t5) - sin(t4 - t5)*cos(t1))*sin(t6) - 1.0*sin(t1)*sin(t2 + t3)*cos(t6),
        1.0*sin(t1)*sin(t4 - t5)*cos(t2 + t3) + 1.0*cos(t1)*cos(t4 - t5)],
       [-1.0*sin(t6)*cos(t2 + t3) - 1.0*sin(t2 + t3)*cos(t6)*cos(t4 - t5),
        1.0*sin(t6)*sin(t2 + t3)*cos(t4 - t5) - 1.0*cos(t6)*cos(t2 + t3),
        -1.0*sin(t2 + t3)*sin(t4 - t5)]], dtype=object)
Position:
  x: 1.0*a2*cos(t1)*cos(t2) + 1.0*a3*cos(t1)*cos(t2 + t3) - 1.0*d3*sin(t1) - 1.0*d4*sin(t2 + t3)*cos(t1)
  y: 1.0*a2*sin(t1)*cos(t2) + 1.0*a3*sin(t1)*cos

In [52]:
cos(pi/2)

0


-----------

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.